In this step, we create a database or reuse one that is already created. We create a table for each data source joining the tables of each source from the formatted zone database.

Before processing any new tables, we remove previously created tables to avoid passing duplicates to the exploitation zone.

In [ ]:
import duckdb
import pandas as pd

sources = ["crimesTables", "pricesTables"]

def formattedToTrusted():
    # Connect to formatted database
    conForm = duckdb.connect(database='../data/formatted/crimesPrices.db')


    # Connect to trusted database
    conTrus = duckdb.connect(database='../data/trusted/crimesPrices.db')

    # Delete any existing tables
    tables = conTrus.execute("SHOW TABLES").df()
    for table in tables['name']:
        conTrus.execute("DROP TABLE " + table)
    
    for source in sources:
        temp = pd.DataFrame()
        with open("../data/formatted/"+source+".txt") as f:
            tables = f.readlines()
        
        # If we have new tables to process
        if len(tables) > 0:
            # Read table names files
            for table in tables:
                temp = pd.concat([temp, conForm.table(table.strip()).to_df()])
            
            # Create a new table with sourceName without "Tables"
            newTable = source[0:-6]
            conTrus.execute('CREATE OR REPLACE TABLE {0} AS SELECT * FROM temp'.format(newTable))

    conForm.close()
    conTrus.close()